# Homework2

## Monthly Retention Rate

### Anna Tigranyan

#### Libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

#### Importing data

In [2]:
df = pd.read_csv("online_retail_II.csv")

df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [3]:
df = df[["InvoiceDate","Customer ID","Country"]]
df.head()

,InvoiceDate,Customer ID,Country
0,2009-12-01 07:45:00,13085.0,United Kingdom
1,2009-12-01 07:45:00,13085.0,United Kingdom
2,2009-12-01 07:45:00,13085.0,United Kingdom
3,2009-12-01 07:45:00,13085.0,United Kingdom
4,2009-12-01 07:45:00,13085.0,United Kingdom


In [4]:
df["InvoiceDate"] = pd.to_datetime(df['InvoiceDate']).dt.date

#### Country filter function

In [5]:
def get_country(data,column,country_name):
    return data.loc[data[column] == country_name]

# For example we wont to calculate monthly retention only for customers from "United Kingdom"  
df = get_country(df,"Country","United Kingdom")

df.head()

,InvoiceDate,Customer ID,Country
0,2009-12-01,13085.0,United Kingdom
1,2009-12-01,13085.0,United Kingdom
2,2009-12-01,13085.0,United Kingdom
3,2009-12-01,13085.0,United Kingdom
4,2009-12-01,13085.0,United Kingdom


In [6]:
def get_month(x): return dt.datetime(x.year, x.month, x.day) 

df['month'] = df['InvoiceDate'].apply(get_month) 
# Grouping by customer Id and select the month value
grouping = df.groupby('Customer ID')['month'] 
# Assigning a minimum month value to the dataset
df['start_date'] = grouping.transform('min')

df.head()

,InvoiceDate,Customer ID,Country,month,start_date
0,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
1,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
2,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
3,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01
4,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01


In [7]:
def get_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month,day

# Getting the integers for date parts
year, month, day = get_int(df, 'month')
year_start, month_start, day_start = get_int(df, 'start_date')

In [8]:
years_diff = year - year_start   #  Get the  difference in years 
months_diff = month - month_start #  Get the  difference in months

df['Index'] = years_diff * 12 + months_diff
df.head()

,InvoiceDate,Customer ID,Country,month,start_date,Index
0,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
1,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
2,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
3,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0
4,2009-12-01,13085.0,United Kingdom,2009-12-01,2009-12-01,0.0


In [9]:
grouping = df.groupby(['start_date', 'Index']) # Counting daily active user from each chort
data = grouping['Customer ID'].apply(pd.Series.nunique) # Counting number of unique customer Id's falling in each group
data = data.reset_index()
 # Assigning column names to the dataframe created above
counts = data.pivot(index='start_date',
                                 columns ='Index',
                                 values = 'Customer ID')
data.head()

,start_date,Index,Customer ID
0,2009-12-01,0.0,107
1,2009-12-01,1.0,49
2,2009-12-01,2.0,48
3,2009-12-01,3.0,51
4,2009-12-01,4.0,42


#### Monthly retention for UK

In [10]:
sizes = counts.iloc[:,0]
retention = counts.divide(sizes, axis=0)

retention.round(3)*100 # Coverting the retention rate into percentage 

Index,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,...,15.0,16.0,17.0,18.0,19.0,20.0,21.0,22.0,23.0,24.0
start_date,,,,,,,,,,,,,,,,,,,,,
2009-12-01,100.0,45.8,44.9,47.7,39.3,43.0,50.5,39.3,35.5,50.5,...,37.4,30.8,35.5,35.5,37.4,33.6,39.3,34.6,47.7,27.1
2009-12-02,100.0,42.7,31.5,44.9,36.0,34.8,47.2,39.3,36.0,38.2,...,25.8,28.1,34.8,29.2,25.8,28.1,32.6,34.8,43.8,27.0
2009-12-03,100.0,32.0,36.1,54.6,49.5,47.4,46.4,50.5,41.2,43.3,...,37.1,34.0,39.2,36.1,35.1,28.9,38.1,40.2,47.4,16.5
2009-12-04,100.0,31.8,40.9,53.0,43.9,45.5,47.0,42.4,40.9,39.4,...,27.3,31.8,28.8,24.2,33.3,28.8,37.9,22.7,40.9,27.3
2009-12-05,100.0,50.0,41.7,45.8,50.0,45.8,37.5,45.8,29.2,29.2,...,29.2,20.8,29.2,37.5,16.7,20.8,33.3,37.5,20.8,20.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011-12-04,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-05,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2011-12-06,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
